#Chatbot Stock Recommendation
###Team members: Jin Hui Xu, Shikha C. Patel, Jhancy Amarsingh

This notebook is part of the DATA 690 NLP Project for building the investment chatbot.

One of the main features of the investment chatbot is to provide stock recommendations to users. In order to generate a list of recommended stocks, we applied the RSI metric to each stock.

The relative strength index (RSI) is a momentum indicator used in technical analysis that measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset. The RSI is displayed as an oscillator (a line graph that moves between two extremes) and can have a reading from 0 to 100.


The recommendation model will use two years of historical data of each stock to calculate the RSI indicator, and it will recommend a list of low RSI stocks which means they meet a condition where an asset has traded lower at price and has the potential for a price bounce.  

Besides, the recommendation model can also recommend a list of stocks based on the stock volume, it will generate a list of high-volume stocks for recommendations if the users choose recommendations based on the volume option.

In [1]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 7.7 MB/s 
     |████████████████████████████████| 63 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import datetime
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np

##RSI Model

In [4]:
# Using yahoo finance API download historical stock values
start = datetime.date.today() - relativedelta(years=2)
end = datetime.date.today() 
symbolscsv = pd.read_csv('symbols.csv')
symbols = symbolscsv['symbols'].values.tolist()

stock_data = yf.download(symbols,start=start,end = end, interval='1d')
stock_data = stock_data.dropna(how='all').dropna(axis=1)


[*********************100%***********************]  502 of 502 completed


In [5]:
#--------------------
stock_close = stock_data["Close"]
Stock_Close_ForRSI = stock_close.iloc[-15:,:]
SP500_diff = pd.DataFrame()
SP500_diff["Date"] = Stock_Close_ForRSI.index
SP500_diff = SP500_diff.set_index("Date")
for col in Stock_Close_ForRSI.columns:
    SP500_diff[col] = Stock_Close_ForRSI[col].diff()
SP500_gain = SP500_diff.clip(lower=0)
SP500_loss = -1 * SP500_diff.clip(upper=0)

SP500_AvgGain = SP500_gain.rolling(window=14).mean()
SP500_AvgLoss = SP500_loss.rolling(window=14).mean()
SP500_RSI = pd.concat([SP500_AvgGain.iloc[-1], SP500_AvgLoss.iloc[-1]], axis=1)
SP500_RSI.columns = ["AvgGain","AvgLoss"]
SP500_RSI["rs"] = round(SP500_RSI["AvgGain"]/SP500_RSI["AvgLoss"],2)
SP500_RSI["rsi"] = 100 - (100 / (1+SP500_RSI["rs"]))
SP500_RSI["Sign"] = [None]*SP500_RSI.shape[0]
SP500_RSI.loc[ np.logical_and(SP500_RSI['rsi'].ge(70), np.less_equal(SP500_RSI['rsi'],100)),'Sign'] = 'OverBought'
SP500_RSI.loc[
 np.logical_and(SP500_RSI['rsi'].ge(0), np.less_equal(SP500_RSI['rsi'], 30)), 
 'Sign'
] = 'OverSold'
SP500_OverBought = SP500_RSI[SP500_RSI["Sign"]=="OverBought"]
SP500_OverSold = SP500_RSI[SP500_RSI["Sign"]=="OverSold"]


In [6]:
SP500_OverBought = SP500_OverBought.sort_values(by=['rsi'] , ascending=False)
SP500_OverSold = SP500_OverSold.sort_values(by=['rsi'] , ascending=True)

In [7]:
SP500_OverBought = SP500_OverBought.rename_axis('Stock').reset_index()

In [8]:
SP500_OverBought

,Stock,AvgGain,AvgLoss,rs,rsi,Sign
0,MHK,2.051429,0.761429,2.69,72.899729,OverBought
1,UAL,0.973571,0.363571,2.68,72.826087,OverBought
2,WDC,0.698571,0.279286,2.50,71.428571,OverBought
3,HST,0.305000,0.124286,2.45,71.014493,OverBought
4,DAL,0.760714,0.311429,2.44,70.930233,OverBought
5,DOW,0.590714,0.246428,2.40,70.588235,OverBought
6,LUV,0.681428,0.290000,2.35,70.149254,OverBought
7,KMB,1.590000,0.679286,2.34,70.059880,OverBought


In [9]:
SP500_OverSold = SP500_OverSold.rename_axis('Stock').reset_index()

In [10]:
SP500_OverSold

,Stock,AvgGain,AvgLoss,rs,rsi,Sign
0,WST,0.230000,7.267142,0.03,2.912621,OverSold
1,NEE,0.030715,1.107143,0.03,2.912621,OverSold
2,D,0.061429,0.542857,0.11,9.909910,OverSold
3,PNW,0.081429,0.627143,0.13,11.504425,OverSold
4,NFLX,2.004286,13.827143,0.14,12.280702,OverSold
...,...,...,...,...,...,...
133,PAYX,0.640714,1.537857,0.42,29.577465,OverSold
134,DTE,0.330715,0.781430,0.42,29.577465,OverSold
135,PSA,1.915000,4.590001,0.42,29.577465,OverSold
136,MCO,1.358571,3.230713,0.42,29.577465,OverSold


Save our recommendation results to file for the chatbot webhook to use

In [11]:
SP500_OverSold.to_csv('stock_oversold.csv')
SP500_OverBought.to_csv('stock_overbought.csv')

## Volume Model

In [19]:
stock_volume = stock_data["Volume"]

In [20]:
days = len(stock_volume)

In [21]:
stock_volume.loc['average_volume'] = stock_volume.sum() / days

In [22]:
stock_volume = pd.DataFrame(stock_volume.loc['average_volume']).sort_values(by=['average_volume'] , ascending=False)
stock_volume = stock_volume.rename_axis('Stock').reset_index()

In [23]:
stock_volume

,Stock,average_volume
0,AAPL,1.086929e+08
1,F,7.896534e+07
2,AMD,6.008522e+07
3,T,5.683287e+07
4,BAC,5.437034e+07
...,...,...
493,TECH,2.303548e+05
494,AZO,2.041093e+05
495,BIO,1.875960e+05
496,MTD,1.133206e+05


Save our recommendation results to file for the chatbot webhook to use

In [24]:
stock_volume.to_csv('stock_high_volume.csv')